# Broad Overview of `onepower`

In this tutorial/demo, we provide a broad overview of the way `onepower` works, and its features. This tutorial will go into a little more depth (without exploring more advanced niche features).

## The Package Layout
`onepower` is quite modular, and contains a number of sub-packages concerning each of the (optional) components that go into defining the resulting power spectra:

In [1]:
from onepower import (
    hmi,      # Interacts with hod and halomod and sets the cosmology
    hod,      # Defines the HODs and returns them to the main class, calculates the observable functions as well (SMF/LF)
    bnl,      # Returns the non-linear halo bias function calculated from Dark Quest emulator
    ia,       # Returns the IA amplitudes for 1h and 2h terms and the fourier transform of the intrinsic alingment profile
    pk        # Defines routines that combine the above to obtain the resulting power spectra
)

While each of these modules has tools that can be useful for more advanced usage, the primary point of contact with `onepower` is the `Spectra` object, which essentially contains all the working of the full package. This lives in the `pk` submodule, but can be imported from the top level:

In [2]:
from onepower import Spectra

## Frameworks -- Caching and Updating

Each main entrypoint class in `onepower` (this includes `HaloModelIngredients`, `NonLinearHaloBias`, `HOD`, ...) is what we call a `Framework`. This functionality is borrowed from the `hmf` and `halomod` directly by calling the relevant function from those packages. What this allows us is to have each of these objects to offer a number of similar points of functionality. Here, we'll demonstrate a few of these bits of functionality on the `HaloModelIngredients` class, but it should be remembered that they are the same for all of these.

In [3]:
from onepower import HaloModelIngredients

In [4]:
hod = HaloModelIngredients()

The first common point is that each of the frameworks has defaults for all of its parameters, and a reasonable object can be created by passing no parameters, as we just did.

We can, like any object in Python, get some help with what parameters are available by using `help`:

In [5]:
help(HaloModelIngredients)

Help on class HaloModelIngredients in module onepower.hmi:

class HaloModelIngredients(CosmologyBase)
 |  HaloModelIngredients(*args, **kwargs)
 |  
 |  A class to compute various ingredients for the halo model.
 |  This includes halo mass functions, bias models, halo profiles, and concentration models.
 |  Based on the hmf and halomod packages.
 |  
 |  Parameters:
 |  -----------
 |  k_vec : array_like, optional
 |      Array of wavenumbers.
 |  lnk_min : float, optional
 |      Minimum natural log of wavenumber (for hmf).
 |  lnk_max : float, optional
 |      Maximum natural log of wavenumber (for hmf).
 |  dlnk : float, optional
 |      Spacing in natural log of wavenumber (for hmf).
 |  Mmin : float, optional
 |      Minimum halo mass (for hmf).
 |  Mmax : float, optional
 |      Maximum halo mass (for hmf).
 |  dlog10m : float, optional
 |      Spacing in log10 of halo mass (for hmf).
 |  mdef_model : str, optional
 |      Mass definition model (for hmf).
 |  hmf_model : str, opt

That's a lot of help! You can also consult the API documentation. However, since many of the parameters to `HaloModelIngredients` merely get passed through to `CosmologyBase` and to the `hmf` / `halomod`, they get lost in this documentation. You can get a list of all possible parameters for a framework like this:

In [6]:
HaloModelIngredients.get_all_parameter_defaults(recursive=False)

{'z_vec': array([0.        , 0.21428571, 0.42857143, 0.64285714, 0.85714286,
        1.07142857, 1.28571429, 1.5       , 1.71428571, 1.92857143,
        2.14285714, 2.35714286, 2.57142857, 2.78571429, 3.        ]),
 'h0': 0.7,
 'omega_c': 0.25,
 'omega_b': 0.05,
 'w0': -1.0,
 'wa': 0.0,
 'n_s': 0.9,
 'tcmb': 2.7255,
 'Neff': 3.044,
 'm_nu': 0.06,
 'sigma_8': 0.8,
 'log10T_AGN': 7.8,
 'hmcode_ingredients': None,
 'k_vec': array([1.00000000e-04, 1.20450354e-04, 1.45082878e-04, 1.74752840e-04,
        2.10490414e-04, 2.53536449e-04, 3.05385551e-04, 3.67837977e-04,
        4.43062146e-04, 5.33669923e-04, 6.42807312e-04, 7.74263683e-04,
        9.32603347e-04, 1.12332403e-03, 1.35304777e-03, 1.62975083e-03,
        1.96304065e-03, 2.36448941e-03, 2.84803587e-03, 3.43046929e-03,
        4.13201240e-03, 4.97702356e-03, 5.99484250e-03, 7.22080902e-03,
        8.69749003e-03, 1.04761575e-02, 1.26185688e-02, 1.51991108e-02,
        1.83073828e-02, 2.20513074e-02, 2.65608778e-02, 3.19926714e-02,


Again, you can consult the API docs for information on each one, bu you can also use this special function:

In [7]:
HaloModelIngredients.parameter_info()

z_vec : array_like
    Array of redshifts.

h0 : float
    Hubble parameter (small h).

omega_c : float
    Cold dark matter density parameter.

omega_b : float
    Baryon density parameter.

w0 : float
    Dark energy equation of state parameter.

wa : float
    Dark energy equation of state parameter.

n_s : float
    Spectral index.

tcmb : float
    Temperature of the CMB.

Neff : float
    Effective number of neutrino species.

m_nu : float
    Neutrino mass.

sigma_8 : float
    Amplitude of matter fluctuations on 8 Mpc scales.

log10T_AGN : float
    Log10 of AGN temperature.

hmcode_ingredients : str
    Correction model from Mead et al.

k_vec : array_like
    Array of wavenumbers.

lnk_min : float
    Minimum natural log of wavenumber (for hmf).

lnk_max : float
    Maximum natural log of wavenumber (for hmf).

dlnk : float
    Spacing in natural log of wavenumber (for hmf).

Mmin : float
    Minimum halo mass (for hmf).

Mmax : float
    Maximum halo mass (for hmf).

dlog10m

Almost all of the things that a framework can calculate -- whether they be transfer functions, growth factors or mass functions -- will appear to be attributes of the object. That is, you don't "call" them like functions, but instead just access them like data. In fact, they are lazily calculated as needed, and then stored in memory once calculated. So, for example, let's calculate the matter power spectrum:

In [8]:
power = Spectra()
%time power.power_spectrum_mm

CPU times: user 4min 42s, sys: 4.43 s, total: 4min 46s
Wall time: 1.94 s


This took almost 6 seconds on this system, as it called CAMB in the background to calculate the power spectrum, and DarkQuest Emulator to calculate the non-linear halo bias. However, it is now cached, and if we call it again:

In [9]:
%time power.power_spectrum_mm

CPU times: user 29 μs, sys: 1 μs, total: 30 μs
Wall time: 46.3 μs


It takes less than 1/1000 of a second, as its just accessing memory. More than that, each (non-trivial) quantity that the power spectrum depends on is also cached, so to access the halo mass function:

In [10]:
%time power.dndlnm.max()

CPU times: user 58 μs, sys: 1 μs, total: 59 μs
Wall time: 71.5 μs


np.float64(2.009046249720673)

Also returns the desired result instanty.

# Updating parameters

Similarly as in `hmf` and `halomod` on which we have based most of functionality, we can update the parameters with the `.update()` method. For more information on this consult the documentation of those packages. 
For instance, we can update the HOD model by:

In [11]:
power.update(hod_model = 'Zheng')

Which will update the model and recalculate all the quantities that depend on it. In other words, it will not recalculate the cosmology or halo mass functions, but the HOD and power spectra will be recomputed. If we would to update the cosmological parameters, then a lot of quantities will be recomputed:

In [12]:
power.update(omega_m = 0.5)

We relly on this functionality to vectorize the `hmf` and `halomod` so that the `onepower` can predict all quantities for an array of redshifts as well.